<a href="https://colab.research.google.com/github/aknip/Coding-Cheatsheets/blob/main/Python-Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install icecream dateparser re --quiet

# Debugging

In [ ]:
# do not use print() - use ic() instead !

from icecream import ic
def foo(i):
    return i + 333
ic(foo(123))

ic| foo(123): 456


456

# Strings

In [ ]:
# Langen String im Code mit Backslash auf mehere Zeilen aufteilen
long_string = 'Hier die ersten Wörter\
 und hier die nächsten. Wichtig:\
 Hinter dem Backslash kein SPACE!'


# Die ersten 10 Zeichen eines Strings
short_string = long_string[:10]

# Langen String (ohne Zeilenumbrüche) ausgeben mit Zeilenumbruch bei 80 Zeichen
import textwrap
print(textwrap.fill(long_string, 80))
# Langer String MIT Zeilenumbrüchen
for x in long_string.split('\n'):
  print(textwrap.fill(x, 80))

# Text in Array splitten - Trennzeichen doppelter Zeilenumbruch
paragraphs_array = fulltext.split('\n\n')

# Array mit Texten in einen String konvertieren, doppelter Zeilenumbruch als Trenner
fulltext_from_array = '\n\n'.join(paragraphs_array)

# Zahl in String wandeln, mit 3 führenden Nullen
number_string = str(number).zfill(3) # 007

# Datum / Uhrzeit als String
from datetime import datetime
current_date_time = datetime.now().strftime("%d.%m.%Y, %H:%M:%S")
# 15.08.2022, 17:49:46



# Templating with Strings

In [ ]:
# Nested templates

# Variant 1:
template = 'Hello, {var1} {var2}.'
var1 = 'John'
var2 = 'Doe'
result1 = template.format(var1=var1, var2=var2)
# Hello, John Doe.

# Variant 2:
def fstr(template):
    return eval(f"f'{template}'")
result2 = fstr(template)
# Hello, John Doe.

result3 = f"{template}" # does not work
result4 = f"Hello {var1}" # works

print(result2)

# Format date and time / Measure time

In [ ]:
# Datum aus String erzeugen, für DE und US
# !pip install dateparser
import dateparser
date_string = "Montag, 10. Juli 2023 08:42:25"
date_obj = dateparser.parse(date_string)
# OR: date_obj = dateparser.parse(date_string, languages=['de'])

# Datum / Uhrzeit als string
from datetime import datetime
current_date_time = datetime.now().strftime("%d.%m.%Y, %H:%M:%S")
# 15.08.2022, 17:49:46

# Dauer messen
import time
start = time.time()
print("hello")
time.sleep(1) # waits for 1 sec
end = time.time()
print(end - start)

# Regex

In [ ]:
# Regex Funktionen
``` Python
import re
# Finde Zahl in String, z. B. die "123" im folgenden String:
#    project x/audio/chunks/segment_123.mp3
# https://regex101.com/r/xwBbvc/1
file_number_regex = re.findall("segment_(\d+).mp3", audio_file)
if file_number_regex:
	file_number = file_number_regex[0] # 1st matching group
else:
	print('Nothing found...')

# Another demo
# Find first occurance
input_txt = "This is a demo text. <Find and replace me>. Lorem ipsum."
text_inside_tags = re.findall('(?sm)<(.+)>', input_txt)[0]
print(text_inside_tags)
# Replace using groups
new_text = 'REPLACED!'
replaced_txt = re.sub('(<)(.+)(>)', r'\1' + new_text + r'\3' , input_txt)
print(replaced_txt)
```


# Array

In [ ]:

# Array Funktionen
``` Python
# Loop through array with index (enumerate as "helper" for index)
for index, part in enumerate(my_array):
    print(index,part)

```


# Dictionary

In [ ]:



# Dictionary (ähnlich Javascript-Objekt/JSON)
``` Python
# Creating a Dictionary
Dict = {'Name': 'Geeks', 1: [1, 2, 3, 4]}
print("Creating Dictionary: ")
print(Dict)

# accessing a element using key - without or without default/fallback
print("Accessing a element using key:")
print(Dict['Name'])
print(Dict['Phone'], 'Sorry, no Phone in Dictionary')

# accessing a element using get()
# method
print("Accessing a element using get:")
print(Dict.get(1))

# creation using Dictionary comprehension
myDict = {x: x**2 for x in [1,2,3,4,5]}
print(myDict)

# convert/print Dictionary to JSON in formatted way
print(json.dumps(myDict, sort_keys=True, indent=2))
```


# Files

In [ ]:


# File schreiben, lesen, kopieren, Pfade/Filenamen bearbeiten
``` Python
import io
import os

# Read full file
f= open('testfile.txt','r')
if f.mode == 'r':
      contents =f.read()
      print(contents)
f.close()

# Read file line-by-line
f= open('testfile.txt','r')
if f.mode == 'r':
      f1 = f.readlines()
      for x in f1:
          print(x)
f.close()

# Write new file
f= open('testfile.txt','w+')
for i in range(10):
     f.write('This is line %d\r\n' % (i+1))
f.close()

# Append to file
f= open('testfile.txt','a+')
f.write('this is appended text 1\r\n')
f.write('this is appended text 2\r\n')
f.close()

# Delete file
os.remove('testfile.txt')

# Create folder (if it not exists already)
if not os.path.exists(text_dir):
	os.mkdir(text_dir)

# Copy file
import shutil
shutil.copyfile(original_with_path, target_with_path)

# Get filename from full path
just_filename = os.path.basename(full_path_incl_filenmae)


# ZIP

In [ ]:

# ZIP (Archive, zip, unzip)
``` Python
# 1.
# with shutil
import shutil
# zip
shutil.make_archive('archive-name-without-.zip', 'zip', 'folder-which-should-be-zipped')
# unzip
shutil.unpack_archive('archive.zip', 'destination-folder')

# 2.
# with zipfile - example: Exclude folder for zip
# https://datagy.io/python-zip-unzip-files/
import os
import zipfile
# zip
directory = "folder-which-should-be-zipped"
with zipfile.ZipFile("zipfile.zip", "w") as zip:
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            srcpath = os.path.join(subdir, file)
            if (subdir != 'exclude-folder1') and (subdir.startswith('exclude-folder2') == False):
              #print(subdir)
              dstpath_in_zip = os.path.relpath(srcpath, start=directory)
              with open(srcpath, 'rb') as infile:
                  print(srcpath)
                  zip.writestr(dstpath_in_zip, infile.read())
# unzip
with zipfile.ZipFile('zipfile.zip', 'r') as zip:
    zip.printdir()
    zip.extractall('./')
```


# Office - Powerpoint

In [ ]:


# Powerpoint Editieren
``` Python
# see my Notebook: https://github.com/aknip/Streamlit-Gradio/blob/main/Powerpoint%20and%20PDF%20Generation.ipynb

# Source: "Using Python to Update PowerPoint. Step by step tutorial to edit PowerPoint slides using Python": https://towardsdatascience.com/use-python-to-automate-the-powerpoint-update-4a385acf1243

# Setup:
# pip install python-pptx

# Fonts:
# sudo apt install fonts-open-sans
# sudo apt install fonts-roboto

from pptx import Presentation
my_ppt = Presentation('ppt-input/presentation.pptx')
# do something with python-pptx
my_ppt.save('ppt-output/presentation2.pptx')
```


# Powerpoint (Office) zu PDF Konvertieren
``` Python
# see my Notebook: https://github.com/aknip/Streamlit-Gradio/blob/main/Powerpoint%20and%20PDF%20Generation.ipynb

# Source: "Stackoverflow: Use unoconv on Ubuntu to convert PPTX to PDF": https://stackoverflow.com/a/63664087

# Setup:
# sudo apt install unoconv
# pip install tqdm
	# pip install glob

# Fonts:
# sudo apt install fonts-open-sans
# sudo apt install fonts-roboto

command = 'unoconv -f pdf -o "pdf-output/presentation.pdf" "ppt-input/presentation.pptx" '
os.system(command)
```


# Office - Word - Markdown

In [ ]:

# Markdown / Word konvertieren, beide Richtungen (Pandoc / pypandoc)
``` Python
#pip install pypandoc
import pypandoc
# md to docx
docx_file = pypandoc.convert_file('input_file.md', 'docx', outputfile='output_file.docx')
# docx to md
md_file = pypandoc.convert_file('input_file.docx', 'md', outputfile='output_file.md')

```



# Word zu Markdown konvertieren
- Ideen/Quellen:
	- https://medium.com/geekculture/how-to-easily-convert-word-to-markdown-with-pandoc-4d60878ccc64
	- https://towardsdatascience.com/word-document-to-html-or-markdown-with-python-37db7150258c
	-



# Email Conversion .msg

In [ ]:
!pip install extract-msg
import extract_msg
msg = extract_msg.openMsg("./msg/test-email.msg")
# or (better:)
!python3 -m extract_msg "msg/test-email.msg" --out msg